# Phase 2B: 원본 소스 데이터 검증 노트북

⚠️ **이 노트북은 source_generator.ipynb 실행 후 계속 실행하세요**

이 노트북은 생성된 원본 소스 데이터를 검증하고 보완합니다.

## 검증 항목
- 필수 필드 존재 여부
- 데이터 타입 확인
- 텍스트 길이 검증
- 카테고리/소스 타입 유효성 확인

## 사용 방법
source_generator.ipynb 실행 후 이 노트북의 셀들을 순차적으로 실행하세요.

## 섹션 1: 패키지 설치

In [ ]:
%pip install pandas python-dotenv

## 섹션 2: 라이브러리 Import

In [ ]:
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict

print("✅ 라이브러리 Import 완료")

## 섹션 3: 데이터 스키마 정의

In [ ]:
# ========================================
# 데이터 스키마
# ========================================

VALID_SOURCE_TYPES = [
    "michelin_review",
    "blueribbon_review",
    "chef_interview",
    "course_description",
    "brand_philosophy",
    "space_ambiance",
    "tradition_technique"
]

VALID_CATEGORIES = [
    "fine_dining",
    "mid_price",
    "waiting_hotplace"
]

def validate_source_data(data: Dict) -> tuple[bool, str]:
    """
    소스 데이터 유효성 검증
    
    Returns:
        (is_valid, error_message)
    """
    # 필수 필드 확인
    required_fields = ["shop_seq", "shop_name", "category", "sources"]
    for field in required_fields:
        if field not in data:
            return False, f"필수 필드 누락: {field}"
    
    # 카테고리 확인
    if data["category"] not in VALID_CATEGORIES:
        return False, f"유효하지 않은 카테고리: {data['category']}"
    
    # sources 리스트 확인
    if not isinstance(data["sources"], list):
        return False, "sources는 리스트여야 합니다"
    
    if len(data["sources"]) == 0:
        return False, "최소 1개 이상의 소스가 필요합니다"
    
    # 각 소스 검증
    for idx, source in enumerate(data["sources"]):
        # 필수 필드
        source_required = ["source_type", "text"]
        for field in source_required:
            if field not in source:
                return False, f"소스 {idx}번: 필수 필드 누락: {field}"
        
        # source_type 확인
        if source["source_type"] not in VALID_SOURCE_TYPES:
            return False, f"소스 {idx}번: 유효하지 않은 source_type: {source['source_type']}"
        
        # 텍스트 길이 확인
        if len(source["text"].strip()) < 50:
            return False, f"소스 {idx}번: 텍스트가 너무 짧습니다 (최소 50자)"
    
    return True, "검증 통과"


print("✅ 데이터 스키마 정의 완료")
print(f"   - 유효한 소스 타입: {', '.join(VALID_SOURCE_TYPES)}")
print(f"   - 유효한 카테고리: {', '.join(VALID_CATEGORIES)}")

## 섹션 4: 데이터 로드

이전 노트북(source_generator.ipynb)에서 전달된 `collected_data`를 사용합니다.

In [ ]:
# ========================================
# 데이터 로드
# ========================================

# source_generator.ipynb에서 전달된 collected_data 사용
if 'collected_data' not in globals():
    print("❌ collected_data가 없습니다!")
    print("   source_generator.ipynb를 먼저 실행하세요.")
    COLLECTED_SOURCES = []
else:
    COLLECTED_SOURCES = collected_data
    print(f"✅ 데이터 로드 완료: {len(COLLECTED_SOURCES)}개 매장")
    
    for shop_data in COLLECTED_SOURCES:
        num_sources = len(shop_data["sources"])
        total_chars = sum(len(s["text"]) for s in shop_data["sources"])
        print(f"   - {shop_data['shop_name']}: {num_sources}개 소스, 총 {total_chars:,}자")

## 섹션 5: 데이터 검증

In [ ]:
# ========================================
# 데이터 검증
# ========================================

print("🔍 데이터 검증 중...\n")
print("=" * 80)

validation_results = []
all_valid = True

for idx, shop_data in enumerate(COLLECTED_SOURCES, 1):
    shop_name = shop_data.get("shop_name", f"매장{idx}")
    
    print(f"\n[{idx}] {shop_name}")
    print("─" * 80)
    
    is_valid, message = validate_source_data(shop_data)
    
    if is_valid:
        print(f"✅ {message}")
        
        # 상세 정보
        num_sources = len(shop_data["sources"])
        total_chars = sum(len(s["text"]) for s in shop_data["sources"])
        source_types = [s["source_type"] for s in shop_data["sources"]]
        
        print(f"   - 카테고리: {shop_data['category']}")
        print(f"   - 소스 개수: {num_sources}개")
        print(f"   - 총 텍스트: {total_chars:,}자")
        print(f"   - 소스 타입: {', '.join(source_types)}")
        
        validation_results.append({
            "shop_name": shop_name,
            "valid": True,
            "num_sources": num_sources,
            "total_chars": total_chars
        })
    else:
        print(f"❌ {message}")
        all_valid = False
        
        validation_results.append({
            "shop_name": shop_name,
            "valid": False,
            "error": message
        })

print("\n" + "=" * 80)

if all_valid:
    print("\n✅ 모든 데이터 검증 통과")
    print(f"   - 총 {len(COLLECTED_SOURCES)}개 매장")
    print(f"   - 총 {sum(r['num_sources'] for r in validation_results)}개 소스")
    print(f"   - 총 {sum(r['total_chars'] for r in validation_results):,}자")
else:
    print("\n❌ 일부 데이터에 오류가 있습니다. 위의 메시지를 확인하세요.")

## 섹션 6: JSON 파일로 저장

In [ ]:
# ========================================
# 검증 완료
# ========================================

if all_valid:
    # 검증된 데이터를 collected_data에 다시 할당
    collected_data = COLLECTED_SOURCES
    
    print("✅ 모든 데이터 검증 통과")
    print(f"   - 총 {len(COLLECTED_SOURCES)}개 매장")
    print(f"   - 총 {sum(r['num_sources'] for r in validation_results)}개 소스")
    print(f"   - 총 {sum(r['total_chars'] for r in validation_results):,}자")
    print("\n📌 다음 노트북을 실행하세요:")
    print("   → source_indexer.ipynb (벡터 DB 인덱싱)")
else:
    print("❌ 일부 데이터에 오류가 있습니다. 위의 메시지를 확인하세요.")

## 섹션 7: 카테고리별 통계 (선택)

In [ ]:
# 카테고리별 통계
if all_valid:
    print("📊 카테고리별 통계\n")
    print("=" * 80)
    
    for category in VALID_CATEGORIES:
        category_data = [s for s in COLLECTED_SOURCES if s["category"] == category]
        
        if category_data:
            print(f"\n📌 {category}")
            print("─" * 80)
            print(f"   - 매장 수: {len(category_data)}개")
            
            total_sources = sum(len(s["sources"]) for s in category_data)
            print(f"   - 총 소스: {total_sources}개")
            
            # 소스 타입별 개수
            source_type_counts = {}
            for shop in category_data:
                for source in shop["sources"]:
                    stype = source["source_type"]
                    source_type_counts[stype] = source_type_counts.get(stype, 0) + 1
            
            print(f"   - 소스 타입별:")
            for stype, count in sorted(source_type_counts.items()):
                print(f"      • {stype}: {count}개")
    
    print("\n" + "=" * 80)

---

# 🔗 source_indexer.ipynb로 이동하세요